<img src="../../images/qiskit-heading.gif" alt="Note: In order for images to show up in this jupyter notebook you need to select File => Trusted Notebook" width="500 px" align="left">

# Tools for Monitoring Backends and Jobs

In this section, we will learn how to monitor the status of jobs submitted to devices and simulators (collectively called backends), as well as discover how to easily query backend details and view the collective state of all the backends available to you.

## Loading the Monitoring Tools

First, let us load the default qiskit routines, and register our IBMQ credentials.

In [1]:
from qiskit import *
IBMQ.load_accounts(hub=None)

Functions for monitoring jobs and backends are here:

In [2]:
from qiskit.tools.monitor import job_monitor, backend_monitor, backend_overview

## Tracking Job Status

Many times a job(s) submitted to the IBM Q network can take a long time to process, e.g. jobs with many circuits and/or shots, or may have to wait in queue for other users.  In situations such as these, it is beneficial to have a way of monitoring the progress of a job, or several jobs at once.  As of Qiskit `0.6+` it is possible to monitor the status of a job in a Jupyter notebook, and also in a Python script (verision `0.7+`).

Lets see how to make use of these tools.

### Monitoring the status of a single job

Lets build a simple Bell circuit, submit it to a device, and then monitor its status.

In [3]:
q = QuantumRegister(2)
c = ClassicalRegister(2)
qc = QuantumCircuit(q, c)

qc.h(q[0])
qc.cx(q[0], q[1])
qc.measure(q, c);

Lets grab the least busy backend

In [4]:
from qiskit.providers.ibmq import least_busy
backend = least_busy(IBMQ.backends(filters=lambda x: not x.configuration().simulator))
backend.name()

'ibmq_16_melbourne'

Monitor the job using `job_monitor` in blocking-mode (i.e. using the same thread as the Python interpretor)

In [6]:
job1 = execute(qc, backend)
job_monitor(job1)

Job Status: job has successfully run


It is also possible to monitor the job using `job_monitor` in async-mode (Jupyter notebooks only).  The job will be monitored in a separate thread, allowing you to continue to work in the notebook. For details see: [Jupyter tools for Monitoring jobs and backends](../jupyter/jupyter_backend_tools.ipynb)

### Changing the interval of status updating

By default, the interval at which the job status is checked is every two seconds.  However, the user is free to change this using the `interval` keyword argument in `job_monitor`

In [7]:
job2 = execute(qc, backend)
job_monitor(job2, interval=5)

Job Status: job has successfully run


## Backend Details

So far we have been executing our jobs on a backend, but we have explored the backends in any detail.  For example, we have found the least busy backend, but do not know if this is the best backend with respect to gate errors, topology etc.  It is possible to get detailed information for a single backend by calling `backend_monitor`:

In [5]:
backend_monitor(backend)

ibmq_16_melbourne
Configuration
-------------
    n_qubits: 14
    operational: True
    status_msg: active
    pending_jobs: 0
    basis_gates: ['u1', 'u2', 'u3', 'cx', 'id']
    local: False
    simulator: False
    max_shots: 8192
    description: 14 qubit device
    max_experiments: 75
    online_date: 2018-11-06T05:00:00+00:00
    url: None
    backend_version: 1.0.0
    credits_required: True
    memory: False
    conditional: False
    open_pulse: False
    sample_name: albatross
    coupling_map: [[1, 0], [1, 2], [2, 3], [4, 3], [4, 10], [5, 4], [5, 6], [5, 9], [6, 8], [7, 8], [9, 8], [9, 10], [11, 3], [11, 10], [11, 12], [12, 2], [13, 1], [13, 12]]
    n_registers: 1
    backend_name: ibmq_16_melbourne
    allow_q_object: True

Qubits [Name / Freq / T1 / T2 / U1 err / U2 err / U3 err / Readout err]
-----------------------------------------------------------------------
    Q0 / 5.10005 GHz / 67.38168 µs / 20.8927 µs / 0.0 / 0.00157 / 0.00313 / 0.0447
    Q1 / 5.23867 GHz / 38.

Or, if we are interested in a higher-level view of all the backends available to us, then we can use `backend_overview()`

In [6]:
backend_overview()

ibmq_16_melbourne           ibmqx4
-----------------           ------
Num. Qubits:  14            Num. Qubits:  5
Pending Jobs: 0             Pending Jobs: 40
Least busy:   True          Least busy:   False
Operational:  True          Operational:  True
Avg. T1:      55.1          Avg. T1:      50.9
Avg. T2:      69.6          Avg. T2:      25.3





There are also Jupyter magic equivalents that give more detailed information, as demonstrated here: [Jupyter tools for Monitoring jobs and backends](../jupyter/jupyter_backend_tools.ipynb)